In [26]:
# load model and predict
import torch
import pandas as pd
import numpy as np
from module import AgeGaitModule,AgeGaitModel

In [27]:
module = AgeGaitModule(AgeGaitModel())
module.load_state_dict(torch.load("model.ckpt")["state_dict"])
module.eval()

AgeGaitModule(
  (train_acc): BinaryAccuracy()
  (valid_acc): BinaryAccuracy()
  (model): AgeGaitModel(
    (conv1): Conv1d(6, 20, kernel_size=(3,), stride=(1,), padding=same)
    (Flatten): Flatten(start_dim=1, end_dim=-1)
    (linear): Linear(in_features=2000, out_features=200, bias=True)
    (linear2): Linear(in_features=200, out_features=1, bias=True)
  )
)

In [73]:
import os
import torch.utils.data as data


data_folder = "AutomaticExtractionData_IMUZCenter"
ids = [file.split('_')[1][2:] for file in os.listdir(data_folder)]
# use 20% of training data for validation
train_set_size = int(len(ids) * 0.8)
valid_set_size = len(ids) - train_set_size

seed = torch.Generator().manual_seed(42)
train_set_id, valid_set_id = data.random_split(ids, [train_set_size, valid_set_size], generator=seed)
train_set = [file for file in os.listdir(data_folder) if file.split('_')[1][2:] in train_set_id ]
valid_set = [file for file in os.listdir(data_folder) if file.split('_')[1][2:] in valid_set_id ]

In [94]:
# read label file
label_file = pd.read_csv("IDGenderAgelist.csv",index_col=0)
def predict(file):
    seqs = []
    id = file.split('_')[-3][2:]
    label = label_file["Gender(0:Female;1:Male)"][int(id)]
    seq = pd.read_csv(file,skiprows=2,header=None).to_numpy()
    for i in range(len(seq)//50-5):
        # print(i*50,(i+6)*50)
        seqs.append(np.fft.fft(seq[i*50:(i+6)*50],n=100,axis=0))
    
    seqs = torch.from_numpy(np.array(seqs)).float()
    # print(seqs.shape,len(seq))
    with torch.no_grad():
        out = module.model(seqs)
        # average the output
        out = torch.mean(out,dim=0)
        # get value from tensor
        out = round(out.numpy().tolist())
    return out,label

In [89]:
predict("AutomaticExtractionData_IMUZCenter/T0_ID000104_Center_seq0.csv")

torch.Size([4, 100, 6]) 536


(1, 1)

In [95]:
true_count = 0
all_count = 0
for file in valid_set:
    try:
        file = os.path.join(data_folder,file)
        # print(file)
        out,label = predict(file)
        all_count += 1
        if out == label:
            true_count += 1
    except:
        # print("error")
        continue
print("accuracy",true_count/all_count)

accuracy 0.9545454545454546
